In [46]:
import os
import glob
import string
import re
import fileinput
import csv
from datetime import date

import pandas as pd

In [27]:
from striprtf.striprtf import rtf_to_text
import docx2txt

In [28]:
# Supporting 3 standards to start: txt, docx, and rtf.
# Maybe have another function which assumes inputs.

# Textract seems to do all these things for me.

In [29]:
r = re.compile(r'\w+')

In [30]:
punctuation_strip = str.maketrans('', '', string.punctuation)

In [31]:
# define content, open it, and read it in prior to the function

def wordcount(content):
    filetoken = content.translate(punctuation_strip).split(' ')
    wordcount = list(filter(r.match, filetoken))
    return len(wordcount)

In [32]:
def filepull(project_path, filetype='txt', isDirectory=False):
    
    if isDirectory is False:
        if filetype == 'txt':
            filepath = os.path.join(project_path)
            with open(filepath, "r", encoding='utf8') as file:
                return file.read()
            
        elif filetype == 'rtf':
            filepath = os.path.join(project_path)
            with open(filepath, "r", encoding='utf8') as file:
                rtf_file = file.read()
                clean_file = rtf_to_text(rtf_file)
                return clean_file
                
        elif filetype == 'docx':
            filepath = os.path.join(project_path)
            file = docx2txt.process(filepath)
            return file

        else:
            print("error: filepull only accepts 'txt', 'rtf', & 'docx' as args")
        
    if isDirectory is True:
        if filetype == 'txt':
            file_list = []
            for filepath in glob.glob(os.path.join(project_path, '*.' + str(filetype))):
                with open(filepath, "r", encoding='utf8') as file:
                    read_in = file.read()
                    file_list.append(read_in)
            merged_files = (" ").join(file_list)
            return merged_files
        
        elif filetype == 'rtf':
            file_list = []
            for filepath in glob.glob(os.path.join(project_path, '*.' + str(filetype))):
                with open(filepath, "r", encoding='utf8') as file:
                    rtf_file = file.read()
                    clean_file = rtf_to_text(rtf_file)
                    file_list.append(clean_file)
            merged_files = (" ").join(file_list)
            return merged_files    
        
        elif filetype == 'docx':
            file_list = []
            for filepath in glob.glob(os.path.join(project_path, '*.' + str(filetype))):
                file_list = docx2txt.process(filepath)
            merged_files = (" ").join(file_list)
            return file_list

In [33]:
test = r'D:\Documents\Datasets\Testing files\rtf files'

In [34]:
test_text = filepull(test, filetype='rtf', isDirectory=True)

In [35]:
def wordmeta_set(name, target, path, filetype):
    
    df = pd.read_csv('wordcount_meta.csv', index_col='Project Name')
    
    if df.index.str.match('^' + str(name) +'$').any() == True:
        new_row = pd.DataFrame.from_records({
            'Project Name':name, 'Latest Target':target, 'Project Path':path, 'Filetype':filetype
        }, index=[0]).set_index('Project Name')
        df.update(new_row)
        df.to_csv('wordcount_meta.csv', index=True)
        
    else:
        new_row = pd.DataFrame.from_records({
            'Project Name':name, 'Latest Target':target, 'Project Path':path, 'Filetype':filetype
        }, index=[0]).set_index('Project Name')
        df = df.append(new_row)
        df.to_csv('wordcount_meta.csv', index=True)

In [55]:
# test this
def wordcount_update(name, dailywords):
    datestamp = date.today().strftime("%d/%m/%Y")
    
    if os.path.exists(str(name) + '_wordcount.csv') == False:
        colnames = [str('Date'), str('Wordcount'), str('Target')]
        with open(str(name) + '_wordcount.csv', 'w', encoding='cp1252') as file:
            filewriter = csv.writer(file)
            filewriter.writerow(colnames)
    else:
        pass
    meta_df = pd.read_csv('wordcount_meta.csv', index_col='Project Name')
    
    df = pd.read_csv(str(name) + '_wordcount.csv', index_col='Date')
    new_row = pd.DataFrame.from_records(
        {'Date': datestamp, 'Wordcount':dailywords, 'Targets':meta_df.loc[name]['Latest Target']})
    df.combine_first(new_row)
    pd.to_csv(str(name) + '_wordcount.csv', index=True)

In [36]:
path=r'D:\Documents\Datasets\001. ML Projects\Wordcount v2\Scrivener-Wordcount-Tool-master\Sample book\Test Adventure.txt'

In [37]:
wordmeta_set(name='Test Adventure',target=300,path=path,filetype='txt')

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [12]:
"""filenames = ['file1.txt', 'file2.txt', ...]
with open('path/to/output/file', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)"""

"filenames = ['file1.txt', 'file2.txt', ...]\nwith open('path/to/output/file', 'w') as outfile:\n    for fname in filenames:\n        with open(fname) as infile:\n            for line in infile:\n                outfile.write(line)"

In [ ]:
# Make a function to write a project name & current wordcount goal to a .csv
# Make a function to write-in a project's daily wordcount & it's current target